# 配置文件

data_finder 模块使用配置文件查找文件路径。
配置文件中定义可选的文件名和可选文件路径，文件查找函数依次在各个目录中查找对应文件是否存在，如果找到文件则返回该文件路径。

In [6]:
import pandas as pd
import numpy as np
import xarray as xr

from pprint import pprint

## 配置文件格式

一个典型的配置文件如下所示：

```yaml
{% set start_time_string = time_vars.year ~ time_vars.month ~ time_vars.day ~ time_vars.hour %}

file_name: 'gmf.gra.{{ start_time_string }}{{ time_vars.forecast_hour }}.grb2'

paths:
  # HPC2023
  - type: local
    level: archive
    path: '/g3/COMMONDATA/OPER/CEMC/GFS_GMF/Prod-grib/{{ start_time_string }}/ORIG'
```

配置文件格式为 YAML，同时也是一个 Jinja2 模板，所以可以使用 Jinja2 的模板语法。

配置文件主要包含文件名和文件目录两个部分。

**文件名**

`file_name` 包含单个文件名，`file_names` 包含多个文件名列表。

**文件目录**

`paths` 包含文件目录列表，每个目录由目录类型 `type`、目录级别 `level` 和目录路径 `path` 三个字段组成。

## 配置文件内置模板变量与函数

配置文件 Jinja2 模板内置了两个变量和一些函数。

### 变量

`TimeVars` 包含起报时间和预报时效等时间变量信息。

- `start_time`: 起报时间，时间类型 (pd.Timestamp)
- `forecast_time`: 预报时效，时间段类型 (pd.Timedelta)
- `year`: 起报时间，年，4 位整数字符串
- `month`: 起报时间，月，2 位整数字符串
- `day`: 起报时间，日，2 位整数字符串
- `hour`: 起报时间，时，2 位整数字符串
- `minute`: 起报时间，分，2 位整数字符串
- `forecast_hour`: 预报时效，时，3 位整数字符串
- `forecast_minute`: 预报时效，分，2 位整数字符串

In [7]:
from reki.data_finder._util import TimeVars

time_vars = TimeVars(
    start_time=pd.to_datetime("2025-07-04 00:00"),
    forecast_time=pd.to_timedelta("24h"),
)

pprint(time_vars.__dict__)

{'day': '04',
 'forecast_hour': '024',
 'forecast_minute': '00',
 'forecast_time': Timedelta('1 days 00:00:00'),
 'hour': '00',
 'minute': '00',
 'month': '07',
 'start_time': Timestamp('2025-07-04 00:00:00'),
 'year': '2025'}


`QueryVars` 包含检索需要的一些其他变量，通常来自调用函数时额外定义的参数，常用变量如下：

- `obs_time`: 观测资料时间，`TimeVars` 对象
- `storage_base`: 基础目录，例如 `/CMADAAS`

### 函数

模板还支持一些函数用于在模板中进行简单计算：

针对起报时间

- `generate_start_time()`: 生成新的起报时间
- `get_year()`: 年，4 位整数字符串
- `get_month()`: 月，2 位整数字符串
- `get_day()`: 日，2 位整数字符串
- `get_hour()`: 时，2 位整数字符串
- `get_minute()`: 分，2 位整数字符串

针对预报时效

- `generate_forecast_time()`: 生成新的预报时效
- `get_forecast_hour()`: 时，3 位整数字符串
- `get_forecast_minute()`: 分，2 位整数字符串

例如下面的配置文件中，观测资料文件名中使用了当前起报时次前 3 小时的时间作为文件名，所以使用 `generate_start_time()` 函数计算得到新的起报时间。

```yaml
{% set start_time_string = time_vars.year ~ time_vars.month ~ time_vars.day ~ time_vars.hour %}
{% set start_time_4dv = generate_start_time(time_vars.start_time, -3) %}
{% set start_time_4dv_string = get_year(start_time_4dv) ~ get_month(start_time_4dv) ~ get_day(start_time_4dv) ~ get_hour(start_time_4dv) %}

file_name: 'rec_R2CWE_{{ start_time_4dv_string }}{{ query_vars.obs_time.hour }}{{ query_vars.obs_time.minute }}_g.dat'

paths:
  # HPC2023
  - type: local
    level: archive
    path: '/g3/COMMONDATA/OPER/CEMC/GFS_GMF/Obs-prep/{{ start_time_string }}'
```